In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
# 依据conn邻接矩阵生成eigenmaps,用于生成空间嵌入
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy.linalg import eigh
def compute_eigenmaps(adj_mx, k):
    A = adj_mx.copy()
    # row, col = A.nonzero()
    # A[row, col] = A[col, row] = 1  # 0/1 matrix, symmetric

    n_components,labels = connected_components(csr_matrix(A), directed=True, connection='strong', return_labels=True)
    assert n_components == 1  # the graph should be connected

    n = A.shape[0]
    A = zero_diagonals(A)
    # degrees = np.sum(A, axis=1)
    # # 避免除以零，将度为零的节点的度设置为一个小正数（例如1e-6）
    # degrees[degrees == 0] = 1e-6
    # D = degrees**(-1/2)
    # L = np.eye(n) - (A * D).T * D  # normalized Laplacian
    D = np.sum(A, axis=1)**(-1/2)
    L = np.eye(n) - (A * D).T * D  # normalized Laplacian

    _, v = eigh(L)
    print('pre_eigenmaps.shape:',v.shape)
    eigenmaps = v[:, 1:(k + 1)]  # eigenvectors corresponding to the k smallest non-trivial eigenvalues

    return eigenmaps

def zero_diagonals(x):
    y = x.copy()
    y[np.diag_indices_from(y)] = 0

    return y

In [3]:
adj_matrix_filtered = pd.read_pickle('./data/adj_matrix_filtered_6392.pkl')
adj_matrix_filtered.shape

(6392, 6392)

In [4]:
eigenmaps = compute_eigenmaps(adj_matrix_filtered,8)
eigenmaps.shape

pre_eigenmaps.shape: (6392, 6392)


(6392, 8)

In [5]:
with open('./data/eigenmaps_6392.pkl', 'wb') as file:
    pickle.dump(eigenmaps, file)
file.close()

In [6]:
eigenmaps = pd.read_pickle('./data/eigenmaps_6392.pkl')
eigenmaps.shape

(6392, 8)

In [12]:
eigenmaps

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-2.60326865e-18,  7.52061171e-18,  2.24283055e-17, ...,
         1.57494714e-19,  1.90298100e-22, -2.18222009e-19],
       [ 6.03086824e-20, -4.35305909e-19,  5.77166193e-18, ...,
         3.66946949e-31,  3.45280319e-17,  1.91374127e-20],
       ...,
       [-3.26289503e-47, -6.33031331e-18,  6.27803539e-18, ...,
        -2.48133092e-48,  9.14444873e-19,  4.59415420e-22],
       [-3.14692367e-47,  2.23417034e-47, -8.01070072e-48, ...,
         7.24416221e-47, -1.58484216e-18,  3.71507820e-22],
       [-8.90707498e-47,  3.79399966e-48, -1.30242547e-47, ...,
         7.19215508e-48,  1.76444610e-48, -1.52756306e-49]])